## Investigate the Coverage of Nerves in SCKAN/Flatmap Neuron Populations within the Human Scaffold

This code uses the **FMA-to-ILX/UBERON** mapping file `(data/generated/mapped_fma_nerves.csv)`, which is generated using `fma_nerve.ipynb`, along with several other pre-generated files to speed up execution.

Required Files
- `data/generated/mapped_fma_nerves.csv`: Contains mappings of FMA terms from the full-body human nerves to ILX or UBERON terms. Only about 420 out of 992 terms can be programmatically mapped.
- `data/generated/UBERON_FMA_Nerve_Hierarchy.json`: Describes the hierarchical structure of nerve-related terms, including their superclasses and subclasses.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import ast
from collections import defaultdict
import requests
import json
import rdflib
import urllib
from tqdm import tqdm
import numpy as np

from utility import GENERATED_DIR, SOURCE_DIR, get_existing_term


In [3]:
## Loading and preprocess mapped_fma_nerves.csv
human_nerve_df = pd.read_csv(GENERATED_DIR / 'mapped_fma_nerves.csv')

## Load set of human nerve terms (ILX/UBERON/FMA)
human_nerves = {
    **(human_nerve_df.loc[
        human_nerve_df['Term ID'].notna(), ['Term ID', 'Group name']
    ].set_index('Term ID')['Group name'].to_dict()),
    **{
        x: row['Group name']
        for _, row in human_nerve_df.iterrows()
        if isinstance(row['available'], str)
        for x in ast.literal_eval(row['available'])
    }
}

In [4]:
## There are some nerves in SCKAN that should be manually mapped to full-body scaffold map
## The mapping is available in `data/source/manually_mapped_missing_nerves.csv'

tmp_df = pd.read_csv(SOURCE_DIR / 'manually_mapped_missing_nerves.csv')
tmp_df[['Missing nerve', 'Missing label']] = tmp_df['Missing SCKAN nerves'].str.split(': ', expand=True)
tmp_df['Manually mapped nerves'] = tmp_df['Manually mapped nerves'].replace('-', np.nan)
tmp_df['Manually mapped nerves'] = tmp_df['Manually mapped nerves'].str.split(r'[\r\n]+')
tmp_df['Manually mapped labels'] = tmp_df['Manually mapped labels'].str.split(r'[\r\n]+')

manually_mapped_nerves = dict(zip(tmp_df.loc[tmp_df['Manually mapped nerves'].notna(), 'Missing nerve'], tmp_df.loc[tmp_df['Manually mapped nerves'].notna(), 'Manually mapped nerves']))

tmp_df[['Missing nerve', 'Missing label', 'Manually mapped nerves', 'Manually mapped labels']]

,Missing nerve,Missing label,Manually mapped nerves,Manually mapped labels
0,ILX:0793714,mesenteric nerve,NaN,NaN
1,ILX:0793208,white communicating ramus of second thoracic s...,[FMA:11278],[T2 white ramus communicans (White communicati...
2,ILX:0793827,posterior abdominal vagal trunk,NaN,NaN
3,UBERON:0018412,vidian nerve,NaN,NaN
4,ILX:0794916,white ramus communicans,"[FMA:10983, FMA:16418]",[White ramus communicans of thoracic spinal ne...
5,ILX:0794476,abdominal branch of vagus nerve,NaN,NaN
6,ILX:0793219,white communicating ramus of thirteenth thorac...,NaN,NaN
7,ILX:0794959,posterior rami lower cervical nerves,NaN,NaN
8,ILX:0793218,white communicating ramus of twelfth thoracic ...,[FMA:11649],[T12 white ramus communicans (White communicat...
9,ILX:0793210,white communicating ramus of fourth thoracic s...,[FMA:10973],[T4 white ramus communicans (White communicati...


In [5]:
NPO_OWNER = 'SciCrunch'
NPO_REPO = 'NIF-Ontology'
NPO_RAW = f'https://raw.githubusercontent.com/{NPO_OWNER}/{NPO_REPO}'
GEN_NEURONS_PATH = 'ttl/generated/neurons/'
TURTLE_SUFFIX = '.ttl'

# A class to explore the relation between terms. The default now is for sckan-2024-09-21
class SCKANTerms():
    def __init__(self, sckan_version='sckan-2024-09-21', labeled_nerves = dict()):
        self.__g = rdflib.Graph()
        for f in ('../../npo', '../../sparc-community-terms'):
            p = urllib.parse.quote(GEN_NEURONS_PATH + f)
            self.__g.parse(f'{NPO_RAW}/{sckan_version}/{p}{TURTLE_SUFFIX}', format='turtle')

        for k, n in {
            'FMA': rdflib.Namespace('http://purl.org/sig/ont/fma/fma'),
            'UBERON': rdflib.Namespace('http://purl.obolibrary.org/obo/UBERON_'),
            'ILX': rdflib.Namespace('http://uri.interlex.org/base/ilx_'),
            'ilxtr': rdflib.Namespace('http://uri.interlex.org/tgbugs/uris/readable/'),
            'BIRNLEX': rdflib.Namespace('http://uri.neuinfo.org/nif/nifstd/birnlex_')
        }.items():
            self.__g.bind(k, n)

        with open(GENERATED_DIR / 'sckan_nerve_existing_ids.json', 'r') as f:
            self.__dumped_existing_ids = json.load(f)

        self.__labeled_nerves = labeled_nerves


    def get_existing_ids(self, term_id):
        q = f"""
            PREFIX FMA: <http://purl.org/sig/ont/fma/fma>
            PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
            PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>
            PREFIX ILX: <http://uri.interlex.org/base/ilx_>
            PREFIX BIRNLEX: <http://uri.neuinfo.org/nif/nifstd/birnlex_>

            SELECT ?existing WHERE {{
                {{
                    {term_id} ilxtr:hasExistingId ?existing .
                }}
                UNION
                {{
                    ?based ilxtr:hasExistingId {term_id} .
                    ?based ilxtr:hasExistingId ?existing .
                }}
                UNION
                {{
                    {term_id} <http://uri.interlex.org/base/ilx_0738400> ?existing .
                }}
            }}
        """
        existings = (
            [self.__g.qname(row.existing) for row in self.__g.query(q)] +
            list(self.__dumped_existing_ids.get(term_id, [])) +
            list(self.__labeled_nerves.get(self.get_label(term_id), []))
        )
        if not existings:
            existings = get_existing_term(term_id)
        return list(set(existings + [term_id])) if isinstance(existings, list) else [term_id]

    def get_label(self, term_id):
        q = f"""
            PREFIX FMA: <http://purl.org/sig/ont/fma/fma>
            PREFIX ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/>
            PREFIX UBERON: <http://purl.obolibrary.org/obo/UBERON_>
            PREFIX ILX: <http://uri.interlex.org/base/ilx_>
            PREFIX BIRNLEX: <http://uri.neuinfo.org/nif/nifstd/birnlex_>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

            SELECT ?label WHERE {{
                {term_id} rdfs:label ?label .
            }}

        """
        if (rows:= list(self.__g.query(q))):
            return str(rows[0].label).lower().strip()
        else:
            return term_id


In [6]:
# A function to execute a query and return results from PostgreSQL
def execute_cq(query):
    pg_endpoint = "https://mapcore-demo.org/devel/flatmap/v4/competency/"

    url = pg_endpoint + 'query'
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=query, headers=headers)

    if response.status_code in (200, 201):
        return response.json().get('results', [])
    else:
        print(f"Error {response.status_code}: {response.json()}")
        return None

# A class to manage human nerves and SCKAN nerves analysis
class HumanSCKANNerves():
    def __init__(self, source_id, human_nerves: dict, manually_mapped_nerves={}):
        self.__source_id = source_id
        self.__human_nerves = human_nerves
        self.__manually_mapped_nerves = manually_mapped_nerves
        self.__load_uberon_fma_nerve_hierarchy()
        self.__sckan_terms = SCKANTerms(labeled_nerves=self.__label_terms)
        self.__load_sckan_nerves(source_id)

    def __load_sckan_nerves(self, source_id):
        ## Load nerves and their paths from competency query
        ## and then stored them into __path_nerves and __nerve_paths
        # get all sckan nerves and their corresponding neuron population (Query 12)
        query = {
            'query_id': '12',
            'parameters': [{'column': 'source_id', 'value': source_id}]
        }
        query_results = execute_cq(query)

        self.__nerve_paths = defaultdict(list)
        for row in query_results['values']:
            self.__nerve_paths[row[1]].append(row[3])

        self.__path_nerves = defaultdict(list)
        for nerve, paths_ids in self.__nerve_paths.items():
            for path_id in paths_ids:
                self.__path_nerves[path_id].append(nerve)

        self.__sckan_nerve_existing_id = {
            nerve_id: set(self.__sckan_terms.get_existing_ids(nerve_id) + [nerve_id])
            for nerve_id in tqdm(self.__nerve_paths)
        }

        with open(GENERATED_DIR / 'sckan_nerve_existing_ids.json', 'w') as f:
            json.dump({k:list(v) for k, v in self.__sckan_nerve_existing_id.items()}, f, indent=4)

        for nerve_id, existing_ids in self.__manually_mapped_nerves.items():
            self.__sckan_nerve_existing_id.setdefault(nerve_id, set()).update(existing_ids)

    def __load_uberon_fma_nerve_hierarchy(self):
        ## Loading UBERON and FMA nerve hierarchy
        with open(GENERATED_DIR / 'UBERON_FMA_Nerve_Hierarchy.json', 'r') as f:
            uberon_fma_nerve_hierarcy = json.load(f)
            self.__subclass_to_superclasses = uberon_fma_nerve_hierarcy['subclass_to_superclasses']
            self.__superclass_to_subclasses = uberon_fma_nerve_hierarcy['superclass_to_subclasses']
            self.__term_labels = uberon_fma_nerve_hierarcy['labels']
            self.__label_terms = defaultdict(set)
            for k, vals in self.__term_labels.items():
                for v in vals:
                    self.__label_terms[v].add(k)

    def get_human_subclass_nerves(self, nerve_id) -> dict:
        subclass_nerves = {}
        for existing_id in self.__sckan_nerve_existing_id[nerve_id]:
            for subclass_id, data in self.__superclass_to_subclasses.get(existing_id, {}).items():
                if subclass_id in self.__human_nerves:
                    subclass_nerves[subclass_id] = self.__term_labels.get(subclass_id, subclass_id)
            if existing_id in self.__human_nerves:
                subclass_nerves[existing_id] = self.__human_nerves[existing_id]
        return subclass_nerves


    def covered_nerves(self, use_subclass=False):
        if not use_subclass:
            return set(self.__nerve_paths) & set(self.__human_nerves)

        sckan_nerve_with_subclasses = {}
        for nerve_id in self.__sckan_nerve_existing_id:
            if self.get_human_subclass_nerves(nerve_id):
                sckan_nerve_with_subclasses[nerve_id] = self.__sckan_terms.get_label(nerve_id)
        return sckan_nerve_with_subclasses

    def uncovered_nerves(self, use_subclass=False):
        return {nerve_id:self.__sckan_terms.get_label(nerve_id) for nerve_id in set(self.__nerve_paths) - set(self.covered_nerves(use_subclass))}

    def uncovered_paths(self, use_subclass=False):
        if not use_subclass:
            missing_paths = {p for k in self.uncovered_nerves(use_subclass) for p in self.__nerve_paths[k]}
            return missing_paths

        sckan_nerve_with_subclasses = set(self.covered_nerves(use_subclass))
        missing_paths = {p for k in set(self.__nerve_paths) - sckan_nerve_with_subclasses for p in self.__nerve_paths[k]}
        return missing_paths

    def covered_paths(self, use_subclass=False):
        return set(self.__path_nerves.keys()) - self.uncovered_paths(use_subclass)

    def print_stats(self):
        print(f"SCKAN/Flatmap source: {self.__source_id}")
        print(f"# number of covered nerves: {len(self.covered_nerves())} out of {len(self.__nerve_paths)}")
        print(f"# number of uncovered nerves: {len(self.uncovered_nerves())} out of {len(self.__nerve_paths)}")
        print(f"# number of covered nerve (consider subclasses): {len(self.covered_nerves(True))} out of {len(self.__nerve_paths)}")
        print(f"# number of uncovered nerves (consider subclasses): {len(self.uncovered_nerves(True))} out of {len(self.__nerve_paths)}")
        print(f"# number of covered paths: {len(self.covered_paths())} out of {len(self.__path_nerves)}")
        print(f"# number of uncovered paths: {len(self.uncovered_paths())} out of {len(self.__path_nerves)}")
        print(f"# number of covered paths (consider subclasses): {len(self.covered_paths(True))} out of {len(self.__path_nerves)}")
        print(f"# number of uncovered paths (consider subclasses): {len(self.uncovered_paths(True))} out of {len(self.__path_nerves)}")

    def save_nerve_coverage(self):
        nerves = defaultdict(list)
        for nerve_id in tqdm(self.__nerve_paths):
            nerves['nerve id'] += [nerve_id]
            nerves['label'] += [self.__sckan_terms.get_label(nerve_id)]
            nerves['covered'] += [nerve_id in self.covered_nerves()]
            nerves['subclasses'] += [list(self.get_human_subclass_nerves(nerve_id).keys())]
            nerves['subclass labels'] += [list(self.get_human_subclass_nerves(nerve_id).values())]

        df = pd.DataFrame(nerves)
        df.to_csv(GENERATED_DIR / f'sckan_human_nerve_coverage_{self.__source_id}.csv')

    def save_path_coverage(self):
        paths = defaultdict(list)
        for path_id, nerve_ids in self.__path_nerves.items():
            paths['path id'] += [path_id]
            paths['covered nerve'] += [list(set(self.covered_nerves()) & set(nerve_ids))]
            paths['uncovered nerve'] += [list(set(nerve_ids) - set(self.covered_nerves()))]
            paths['covered nerve (subclasses)'] += [[x for nerve_id in nerve_ids for x in self.get_human_subclass_nerves(nerve_id)]]
            paths['complete (subclasses)'] += [all([len(self.get_human_subclass_nerves(nerve_id)) > 0 for nerve_id in nerve_ids])]
        df = pd.DataFrame(paths)
        df.to_csv(GENERATED_DIR / f'sckan_human_path_coverage_{self.__source_id}.csv')


#### Check the coverage of human nerve on sckan-2024-09-21

In [7]:
# Create an object and print stat
hsn = HumanSCKANNerves('sckan-2024-09-21', human_nerves, manually_mapped_nerves)
hsn.print_stats()

100%|██████████| 116/116 [00:06<00:00, 17.09it/s]


SCKAN/Flatmap source: sckan-2024-09-21
# number of covered nerves: 0 out of 116
# number of uncovered nerves: 116 out of 116
# number of covered nerve (consider subclasses): 86 out of 116
# number of uncovered nerves (consider subclasses): 30 out of 116
# number of covered paths: 0 out of 275
# number of uncovered paths: 275 out of 275
# number of covered paths (consider subclasses): 195 out of 275
# number of uncovered paths (consider subclasses): 80 out of 275


It is clear that when comparing nerve terms in SCKAN directly to those in the full-body human nerve map, there are no exact matches—resulting in zero neuron populations being covered. This is because the nerve terms in SCKAN typically represent higher-level structures, such as the splanchnic nerve (UBERON:0003715), or generic terms like “nerve” (UBERON:0001021). Some terms also lack specified laterality, such as the auriculotemporal nerve (ILX:0793723). In contrast, the full-body human nerve map includes more specific, anatomically detailed terms.

However, after taking into account the subclasses of the SCKAN nerve terms, the coverage improves significantly: 54 out of 105 nerves are now matched, resulting in 145 out of 274 neuron populations being covered.

In [8]:
# Get uncovered neuron population (considering subclass)
hsn.uncovered_paths(use_subclass=True)

{'ilxtr:neuron-type-aacar-12',
 'ilxtr:neuron-type-bolew-unbranched-14',
 'ilxtr:neuron-type-bolew-unbranched-15',
 'ilxtr:neuron-type-bolew-unbranched-20',
 'ilxtr:neuron-type-bolew-unbranched-21',
 'ilxtr:neuron-type-bolew-unbranched-23',
 'ilxtr:neuron-type-bolew-unbranched-8',
 'ilxtr:neuron-type-bromo-1',
 'ilxtr:neuron-type-keast-1',
 'ilxtr:neuron-type-keast-10',
 'ilxtr:neuron-type-keast-11',
 'ilxtr:neuron-type-keast-2',
 'ilxtr:neuron-type-keast-3',
 'ilxtr:neuron-type-keast-4',
 'ilxtr:neuron-type-keast-5',
 'ilxtr:neuron-type-sdcol-b',
 'ilxtr:neuron-type-sdcol-c',
 'ilxtr:neuron-type-sdcol-d',
 'ilxtr:neuron-type-sdcol-f',
 'ilxtr:neuron-type-sdcol-g',
 'ilxtr:neuron-type-sdcol-h',
 'ilxtr:neuron-type-sdcol-q',
 'ilxtr:neuron-type-sdcol-q-prime',
 'ilxtr:neuron-type-splen-5',
 'ilxtr:sparc-nlp/femrep/100',
 'ilxtr:sparc-nlp/femrep/101',
 'ilxtr:sparc-nlp/femrep/38',
 'ilxtr:sparc-nlp/femrep/52',
 'ilxtr:sparc-nlp/femrep/62',
 'ilxtr:sparc-nlp/femrep/63',
 'ilxtr:sparc-nlp/

In [9]:
# Get uncovered nerve (considering subclass)
hsn.uncovered_nerves(use_subclass=True)

{'ILX:0793827': 'posterior abdominal vagal trunk',
 'ILX:0795006': 'suboccipital nerve',
 'ILX:0793822': 'superior ovarian nerve',
 'ILX:0793227': 'gray communicating ramus of thirteenth thoracic nerve',
 'ILX:0793559': 'bladder nerve',
 'UBERON:0034984': 'nerve to quadratus femoris',
 'ILX:0738309': 'internal branch of inferior laryngeal nerve',
 'ILX:0794959': 'posterior rami lower cervical nerves',
 'ILX:0794949': 'recurrent branch of the median nerve',
 'ILX:0794141': 'right cervical vagus nerve',
 'ILX:0739299': 'gray communicating ramus of sixth lumbar nerve',
 'UBERON:0022302': 'short ciliary nerve',
 'ILX:0793219': 'white communicating ramus of thirteenth thoracic spinal nerve',
 'UBERON:0001650': 'hypoglossal nerve',
 'ILX:0793809': 'clitoral cavernous nerve',
 'ILX:0738293': 'ganglioglomerular nerve',
 'ILX:0793632': 'lumbar colonic nerve',
 'UBERON:0018675': 'pelvic splanchnic nerve',
 'ILX:0793563': 'splenic nerve',
 'ILX:0738312': 'aortic arch depressor nerve',
 'ILX:07949

In [10]:
# manually_mapped_nerves['UBERON:0018412']
hsn._HumanSCKANNerves__sckan_nerve_existing_id['UBERON:0018412']
{**hsn._HumanSCKANNerves__sckan_nerve_existing_id, **{'UBERON:0018412':'2'}}['UBERON:0018412']

'2'

In [11]:
# Now it is also possible to save the report of covered nerved and paths in csv files

# Report of the covered SCKAN nerves.
# File: data/generated/sckan_human_nerve_coverage_{source_id}.csv
# Columns:  - `nerve id`: the id of SCKAN nerve
#           - `label`: the nerve label
#           - `covered`: status whether the nerve directly covered (True/False)
#           - `subclasses`: the available nerve subclasses
#           - `subclass labels`: the subclass labels
#
hsn.save_nerve_coverage()

# Report of the covered SCKAN neuron populations.
# File: data/generated/sckan_human_path_coverage_{source_id}.csv
# Columns:  - `path id`: the neuron population id
#           - `covered nerve`: the covered neuron population's nerves
#           - `uncovered nerve`: the uncovered neuron population's nerves
#           - `covered nerve (subclasses)`: the covered neuron population's nerves considering subclasses
hsn.save_path_coverage()

100%|██████████| 116/116 [00:00<00:00, 450.84it/s]


#### Check the coverage of human nerve on male-flatmap 2b76d336-5c56-55e3-ab1e-795d6c63f9c1

In [12]:
# Create an object and print stat
hsn_male = HumanSCKANNerves('2b76d336-5c56-55e3-ab1e-795d6c63f9c1', human_nerves)
hsn_male.print_stats()

100%|██████████| 41/41 [00:01<00:00, 30.41it/s]


SCKAN/Flatmap source: 2b76d336-5c56-55e3-ab1e-795d6c63f9c1
# number of covered nerves: 0 out of 41
# number of uncovered nerves: 41 out of 41
# number of covered nerve (consider subclasses): 25 out of 41
# number of uncovered nerves (consider subclasses): 16 out of 41
# number of covered paths: 0 out of 104
# number of uncovered paths: 104 out of 104
# number of covered paths (consider subclasses): 65 out of 104
# number of uncovered paths (consider subclasses): 39 out of 104


In [13]:
hsn_male.covered_paths(use_subclass=True)

{'ilxtr:neuron-type-aacar-11',
 'ilxtr:neuron-type-aacar-4',
 'ilxtr:neuron-type-bolew-unbranched-10',
 'ilxtr:neuron-type-bolew-unbranched-12',
 'ilxtr:neuron-type-bolew-unbranched-16',
 'ilxtr:neuron-type-bolew-unbranched-17',
 'ilxtr:neuron-type-bolew-unbranched-18',
 'ilxtr:neuron-type-bolew-unbranched-19',
 'ilxtr:neuron-type-bolew-unbranched-4',
 'ilxtr:neuron-type-bolew-unbranched-5',
 'ilxtr:neuron-type-bolew-unbranched-9',
 'ilxtr:neuron-type-bromo-1',
 'ilxtr:neuron-type-bromo-2',
 'ilxtr:neuron-type-keast-12',
 'ilxtr:neuron-type-keast-7',
 'ilxtr:neuron-type-keast-9',
 'ilxtr:neuron-type-pancr-1',
 'ilxtr:neuron-type-sdcol-b',
 'ilxtr:neuron-type-splen-1',
 'ilxtr:neuron-type-splen-2',
 'ilxtr:neuron-type-splen-4',
 'ilxtr:neuron-type-sstom-10',
 'ilxtr:neuron-type-sstom-5',
 'ilxtr:neuron-type-sstom-7',
 'ilxtr:neuron-type-sstom-8',
 'ilxtr:neuron-type-sstom-9',
 'ilxtr:sparc-nlp/kidney/131',
 'ilxtr:sparc-nlp/kidney/133',
 'ilxtr:sparc-nlp/kidney/135',
 'ilxtr:sparc-nlp/k

In [14]:
hsn_male.uncovered_nerves(use_subclass=True)

{'ILX:0738293': 'ganglioglomerular nerve',
 'ILX:0793632': 'lumbar colonic nerve',
 'UBERON:0018675': 'pelvic splanchnic nerve',
 'UBERON:0022302': 'short ciliary nerve',
 'ILX:0738312': 'aortic arch depressor nerve',
 'ILX:0738292': 'middle cervical ganglion - superior cervical ganglion interganglionic segment of the sympathetic chain',
 'ILX:0793559': 'bladder nerve',
 'UBERON:0001650': 'hypoglossal nerve',
 'ILX:0738313': 'rami glomi carotici',
 'ILX:0738291': 't1 - inferior cervical ganglion interganglionic segment of the sympathetic chain',
 'UBERON:0005303': 'hypogastric nerve',
 'ILX:0738309': 'internal branch of inferior laryngeal nerve',
 'ILX:0738290': 'inferior cervical ganglion - middle cervical ganglion interganglionic segment of the sympathetic chain',
 'UBERON:0018412': 'vidian nerve',
 'ILX:0793711': 'communicating branch of zygomatic nerve',
 'ILX:0738308': 'external branch of inferior laryngeal nerve'}